<a href="https://colab.research.google.com/github/OliBomby/osu-diffusion/blob/master/colab/osu_diffusion_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beatmap Style Transfer with osu-diffusion

This notebook is an interactive demo of an osu! beatmap generation model created by OliBomby. This model is only capable of generating hit object coordinates from a given rhythm, spacing, and style. You can upload a beatmap and have the model automatically remap it into any style available.

### Instructions for running:

* Make sure to use a GPU runtime, click:  __Runtime >> Change Runtime Type >> GPU__
* Press ▶️ on the left of each cell to execute the cell
* In the __Choose a Style__ cell, choose a style to map the beatmap in and the desired number of sampling steps
* In the __Upload Beatmap__ cell, choose a .osu file from your computer when prompted
* Remap the beatmap using the __Convert beatmap__ cell (it may take a few minutes depending on the length of the beatmap and the number of sampling steps)

In [1]:
#@title Setup Environment
#@markdown Install osu-diffusion and download the model (may take a few minutes).

!git clone https://github.com/OliBomby/osu-diffusion
%cd osu-diffusion

!git lfs install
!git clone https://huggingface.co/OliBomby/osu-diffusion

!pip install slider

from google.colab import files

import os
import re
from slider import Beatmap
from data_loading import beatmap_to_sequence, split_and_process_sequence, get_beatmap_idx

Cloning into 'osu-diffusion'...
remote: Enumerating objects: 589, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 589 (delta 27), reused 27 (delta 25), pack-reused 555
Receiving objects: 100% (589/589), 692.81 KiB | 24.74 MiB/s, done.
Resolving deltas: 100% (376/376), done.
/content/osu-diffusion
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.1 MB/s eta 0:00:00
Downloading...
From: https://drive.google.com/uc?id=1oX8SPNnyswhaI8euWRkJ10tmncxaGsBG
To: /content/osu-diffusion/DiT-B-0700000.pt
100% 2.73G/2.73G [00:29<00:00, 93.3MB/s]


In [2]:
#@title Upload Beatmap

def upload_beatmap():
  data = list(files.upload().keys())
  if len(data) > 1:
    print('Multiple files uploaded; using only one.')
  return data[0]

input_beatmap = upload_beatmap()

Saving Chogakusei - Kemono ni Naritai! (Kroytz) [Mindmaster wants to be a HARDER Kemono].osu to Chogakusei - Kemono ni Naritai! (Kroytz) [Mindmaster wants to be a HARDER Kemono].osu


In [17]:
#@title Choose a Style

style_id = None

STYLE = "None" #@param["None", "Cheri", "Monstrata", "Sotarks", "wafer", "Nevo", "ScubDomino", "Night Mare", "Other"]
STYLE_TO_ID = {
    "None": None,
    "Cheri": 2592760,
    "Monstrata": 1451282,
    "Sotarks": 1995061,
    "wafer": 3697057,
    "Nevo": 2799753,
    "ScubDomino": 1772923,
    "Night Mare": 1907310,
}

if STYLE == "Other":
    style_id = int(input("Please type the Beatmap ID of the beatmap you want the style of: "))
    beatmap_idx = get_beatmap_idx("beatmap_idx.pickle")
    if style_id not in beatmap_idx:
        print("This beatmap style does not exist in the model. Try choosing a ranked osu! std beatmap from before 2023.")
    else:
        print("Good choice!")
else:
    style_id = STYLE_TO_ID[STYLE]
    print(f"Chosen style: {STYLE}")

NUM_SAMPLING_STEPS = "1000" #@param[250, 1000]
print(f"Chosen step count: {NUM_SAMPLING_STEPS}")


Chosen style: Monstrata
Chosen step count: 1000


In [18]:
#@title Convert Beatmap
#@markdown This may take a few minutes depending on the length of the beatmap
#@markdown file you uploaded and the number of sampling steps.

beatmap = Beatmap.from_path(input_beatmap)

CLEAN_FILENAME_RX = re.compile(r"[/\\?%*:|\"<>\x7F\x00-\x1F]")
filename = f"{beatmap.beatmap_id} {beatmap.artist} - {beatmap.title}"
filename = CLEAN_FILENAME_RX.sub("-", filename)
result_dir = os.path.join(
    "results",
    filename,
    f"{beatmap.beatmap_id} result {style_id} 0.osu",
)

if style_id is not None:
    !python sample.py --beatmap "{input_beatmap}" --ckpt "osu-diffusion/DiT-B-0700000.pt" --style-id {style_id} --num-sampling-steps {NUM_SAMPLING_STEPS}
else:
    !python sample.py --beatmap "{input_beatmap}" --ckpt "osu-diffusion/DiT-B-0700000.pt" --num-sampling-steps {NUM_SAMPLING_STEPS}

print(f"Beatmap successfully saved to {result_dir}")


seq len 1178
100% 1000/1000 [04:05<00:00,  4.07it/s]
Beatmap successfully saved to results/3671513 Chogakusei - Kemono ni Naritai!/3671513 result 1451282 1177.osu


In [19]:
#@title Download Converted Beatmap


files.download(result_dir)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>